### 練習問題2.37
ベクトル${\bf v}=(v_{i})$を数値の列として表現し、  
⾏列 ${\bf m}=(m_{ij})$をベクトル(⾏列の⾏)の列として表現するとする。  
例えば、以下の⾏列は
$$
\left(
    \begin{array}{cccc}
      1 & 2 & 3 & 4 \\
      4 & 5 & 6 & 6 \\
      6 & 7 & 8 & 9
    \end{array}
  \right)
$$  

列((1 2 3 4) (4 5 6 6) (6 7 8 9))として表現される。  
この表現を使うと、列操作によって基本的な⾏列とベクトルの演算を簡潔に表現できる。  
これらの(どんな行列演算の本にも書いてある) 演算は、次のようなものである。  

(dot-product v w)・・・総和$\sum_{i} v_{i}w_{i}$を返す。  
(matrix-\*-vector m v)・・・$t_{i}=\sum_{j} v_{ij}w_{j}$ であるような ベクトル${\bf t}$を返す。  
(matrix-\*-matrix m n)・・・$p_{ij}=\sum_{k}m_{ik}n_{kj}$であるような ⾏列${\bf p}$を返す。  
(transpose m)・・・$n_{ij}=m_{ji}$であるような ⾏列${\bf n}$を返す   

内積は、次のように定義できる。  

    (define (dot-product v w)
       (accumulate + 0 (map * v w)))  
       
ほかの行列演算を計算する以下の手続きについて、欠けた式を補え  
(⼿続きaccumulate-nは練習問題2.36で定義されている)。  

    (define (matrix-*-vector m v)
        (map ⟨??⟩ m))
    (define (transpose mat)
        (accumulate-n ⟨??⟩ ⟨??⟩ mat))
    (define (matrix-*-matrix m n)
        (let ((cols (transpose n)))
            (map ⟨??⟩ m)))

In [1]:
; 集積
(define (accumulate op initial sequence)
  (if (null? sequence) initial
      (op (car sequence) (accumulate op initial (cdr sequence)))))

(define (accumulate-n op init seqs)
    (if (null? (car seqs)) '()
        (cons (accumulate op init (accumulate (lambda (x y)(cons (car x) y)) () seqs))
              (accumulate-n op init (accumulate (lambda (x y)(cons (cdr x) y)) () seqs))
              )
        )
  )

In [2]:
; 内積
(define (dot-product v w)
   (accumulate + 0 (map * v w)))

; 動作確認
(dot-product '(2 3) '(-4 2))

-2

In [3]:
; 回答
; 転置行列
; (define (transpose mat)
;     (accumulate-n ⟨??⟩ ⟨??⟩ mat))

(define (transpose mat)
    (accumulate-n cons () mat))

; 動作確認
(transpose '((1 2 3 4) (4 5 6 6) (6 7 8 9)))

((1 4 6) (2 5 7) (3 6 8) (4 6 9))

In [4]:
; 動作確認
(transpose '((1 2) (4 3) (2 1) (5 3)))

((1 4 2 5) (2 3 1 3))

In [5]:
; 回答
; 行列×ベクトル
;(define (matrix-*-vector m v)
;    (map ⟨??⟩ m))

(define (matrix-*-vector m v)
  (map (lambda (row)(dot-product row v)) m)
  )

; 動作確認
(matrix-*-vector '((1 2) (3 4)) '(2 3))

(8 18)

In [6]:
; 回答
; 行列の積
; (define (matrix-*-matrix m n)
;     (let ((cols (transpose n)))
;        (map ⟨??⟩ m)))

(define (matrix-*-matrix m n)
    (let ((cols (transpose n)))
        (map (lambda (row)
               (map (lambda (col) (dot-product row col)) cols))
             m)
      )
  )

; 動作確認
(matrix-*-matrix '((1 2 3 4) (4 5 6 6) (6 7 8 9)) '((1 2) (4 3) (2 1) (5 3)))

((35 23) (66 47) (95 68))

In [7]:
; 回答
; 行列の積
; 別解(transpose手続き使用)

(define (matrix-*-matrix m n)
    (let ((cols (transpose n)))
      (transpose (map (lambda (col) (matrix-*-vector m col)) cols))
      )
  )

; 動作確認
(matrix-*-matrix '((1 2 3 4) (4 5 6 6) (6 7 8 9)) '((1 2) (4 3) (2 1) (5 3)))

((35 23) (66 47) (95 68))

-------------------------------------------------------------

In [8]:
; 検討のための動作確認
(define m '((1 2 3 4) (4 5 6 6) (6 7 8 9)))
(define n '((1 2) (4 3) (2 1) (5 3)))
(define cols (transpose '((1 2) (4 3) (2 1) (5 3))))

; accumulate-nで乗算・加算を実行する。
(accumulate + 0 (accumulate-n * 1 (list (car m) (car cols))))

35

In [9]:
; 検討のための動作確認
(define (multi-sum row col)
  (accumulate + 0
    (map (lambda (x)(let ((a (car x))(b (car (cdr x))))(* a b)))
        (transpose (list row col))
      )
    )
  )
(multi-sum (car m) (car cols))

35

In [10]:
; 検討のための動作確認
(accumulate + 0
            (map (lambda (x)(let ((a (car x))(b (car (cdr x))))(* a b)))
                 (transpose '((11 13 15 17) (1 2 3 4)))
                 )
    )

150

In [11]:
; 検討のための動作確認
((lambda (x)(accumulate cons
                        ()
                        (map (lambda (y)(transpose (list x y))) cols)
                )
        ) (car m))

(((1 1) (2 4) (3 2) (4 5)) ((1 2) (2 3) (3 1) (4 3)))

In [12]:
; 検討のための動作確認
(map (lambda (x)
       (let ((a (car x))
             (b (car (cdr x))))
         (* a b)
         )
       )
     (transpose '((11 13 15 17) (1 2 3 4)))
     )

(11 26 45 68)

In [13]:
; ボツ回答
; 行列×ベクトル
;(define (matrix-*-vector m v)
;    (map ⟨??⟩ m))

(define (matrix-*-vector m v)
  (map (lambda (x)(accumulate (lambda (a b) a)
                     ()
                     (map (lambda (y)(accumulate + 0 (accumulate-n * 1 (list x y)))) (list v))
            )
        )
    m)
  )

; 動作確認
(matrix-*-vector '((1 2) (3 4)) '(2 3))

(8 18)

In [14]:
; ボツ回答
; 行列の積
; (define (matrix-*-matrix m n)
;     (let ((cols (transpose n)))
;        (map ⟨??⟩ m)))

(define (matrix-*-matrix m n)
    (let ((cols (transpose n)))
        (map (lambda (x)(accumulate cons
                                    ()
                                    (map (lambda (y)(accumulate + 0 (accumulate-n * 1 (list x y)))) cols)
                            )
               )
             m)
      )
  )

; 動作確認
(matrix-*-matrix '((1 2 3 4) (4 5 6 6) (6 7 8 9)) '((1 2) (4 3) (2 1) (5 3)))

((35 23) (66 47) (95 68))